# No Clue

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

# Set up Selenium WebDriver (headless mode for faster execution)
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run without opening a browser
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Automatically detect ChromeDriver
driver = webdriver.Chrome(options=chrome_options)

# Open the webpage
url = "https://shop-noclue.com/collections/all-products"
driver.get(url)

# Wait for JavaScript to load
time.sleep(5)

# Extract all product links
products = []
product_cards = driver.find_elements(By.CSS_SELECTOR, "div.card__content")
visited_cards = set()
# Find product cards
while True:
    try:
        product_cards = driver.find_elements(By.CSS_SELECTOR, "div.card__content")
        break
    except Exception:
        time.sleep(2)
        continue

for i in range(len(product_cards)):
    try:
        product_cards = driver.find_elements(By.CSS_SELECTOR, "div.card__content")  # Refresh elements
        product_link = product_cards[i].find_element(By.CSS_SELECTOR, "a.full-unstyled-link")
        product_url = product_link.get_attribute("href")
        print(product_url)

        # Click product and wait for new page to load
        driver.execute_script("arguments[0].click();", product_link)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))

        # Extract product details
        title = driver.find_element(By.CSS_SELECTOR, "div.product__title h1").text.strip()
        if title in visited_cards:
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card__content")))
            continue
        visited_cards.add(title)
        print(title)

        brand = driver.find_element(By.CSS_SELECTOR, "img.header__heading-logo").get_attribute("alt")
        print(brand)

        # Extract regular price (if available)
        regular_price_element = driver.find_elements(By.CSS_SELECTOR, "div.price__regular span.price-item.price-item--regular")
        regular_price = regular_price_element[0].text.strip() if regular_price_element else ""

        # Extract sale price (if available)
        sale_price_element = driver.find_elements(By.CSS_SELECTOR, "div.price__sale span.price-item.price-item--sale.price-item--last")
        sale_price = sale_price_element[0].text.strip() if sale_price_element else ""

        # If a sale price exists but no regular price, the regular price might be in the crossed-out <s> tag
        if sale_price and not regular_price:
            crossed_out_price_element = driver.find_elements(By.CSS_SELECTOR, "div.price__sale s.price-item.price-item--regular")
            regular_price = crossed_out_price_element[0].text.strip() if crossed_out_price_element else ""

        print(regular_price)
        print(sale_price)
        
        description = driver.find_element(By.CSS_SELECTOR, "div.product__description p").text.strip()
        print(description)
        
        sizes = []
        size_elements = driver.find_elements(By.CSS_SELECTOR, "fieldset input[name='Size']")

        for size_element in size_elements:
            size_value = size_element.get_attribute("value")
            size_class = size_element.get_attribute("class")
            
            if "disabled" not in size_class:  # Only add if it's available
                sizes.append(size_value)

        sizes = ",".join(sizes)
        print("Available Sizes:", sizes)

        available = 0 if sizes == "" else 1
        

        image_url = driver.find_element(By.CSS_SELECTOR, "div.product__media img").get_attribute("src")
        print(image_url)

        # Append data
        products.append({
            "brand": brand,
            "title": title,
            "product_url": product_url,
            "regular_price": regular_price,
            "sale_price":sale_price,
            "description": description,
            "sizes": sizes,
            "available": available,
            "image_url": image_url
        })

        # Go back and wait for products page to reload
        driver.back()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card__content")))

    except Exception as e:
        print(f"Error scraping a product: {e}")
        driver.back()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card__content")))
        continue

# Close Selenium WebDriver
driver.quit()

# Save to CSV
df = pd.DataFrame(products)
print(df.head)
df.to_csv("hedomak_products.csv", index=False)

print("Scraping complete. Data saved!")


# Juvenile

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
# Set up Selenium WebDriver (headless mode for faster execution)
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run without opening a browser
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Automatically detect ChromeDriver
driver = webdriver.Chrome(options=chrome_options)
page_number = 1
products = []
visited_cards = set()

while True:
    # Open the webpage
    url = f"https://juvenileeg.com/collections/all?page={page_number}"
    driver.get(url)

    # Wait for JavaScript to load
    time.sleep(10)

    # Extract all product links

    product_cards = driver.find_elements(By.CSS_SELECTOR, "div.block-inner-inner")


    
    # Find product cards
    while True:
        try:
            product_cards = driver.find_elements(By.CSS_SELECTOR, "div.block-inner-inner")
            break
        except Exception:
            time.sleep(2)
            continue
    print("Ok")
    print(len(product_cards))
    for i in range(len(product_cards)):
        print("Running")
        
        try:
            product_cards = driver.find_elements(By.CSS_SELECTOR, "div.block-inner-inner")  # Refresh elements
            product_link = product_cards[i].find_element(By.CSS_SELECTOR, "a.product-link")
            product_url = product_link.get_attribute("href")
            print(product_url)

            # Click product and wait for new page to load
            driver.execute_script("arguments[0].click();", product_link)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))

            # Extract product details
            time.sleep(5)
            title = driver.find_element(By.CSS_SELECTOR, "h1.title").text.strip()
            if title in visited_cards:
                print("Already Exists")
                driver.back()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block-inner-inner")))
                continue
            visited_cards.add(title)
            if title=="":
                print("Failed")
                driver.quit()
                
            print(title)

            brand = driver.find_element(By.CSS_SELECTOR, "img.logo__image").get_attribute("alt")
            print(brand)

            try:
                sale_price = driver.find_element(By.CSS_SELECTOR, "span.current-price.theme-money").text.strip()
            except:
                sale_price = ""

            try:
                regular_price = driver.find_element(By.CSS_SELECTOR, "span.was-price.theme-money").text.strip()
            except:
                regular_price = sale_price  # If no sale, set regular price to sale price

            # Ensure both prices are the same if no discount is applied
            if not regular_price:
                regular_price = sale_price

            print(regular_price)
            print(sale_price)
            
            try:
                description = driver.find_element(By.CSS_SELECTOR, "div.product-description.rte.cf span").text.strip()
            except:
                try:
                    description = driver.find_element(By.CSS_SELECTOR, "div.product-description.rte.cf p").text.strip()
                except:
                    description = ""
            print(description)
            
            
            # Locate the div containing sizes
            size_buttons = driver.find_elements(By.CSS_SELECTOR, "div.option-selector__btns input.opt-btn")

            sizes = []

            for button in size_buttons:
                # Check if the size is available (does not have 'is-unavailable' class)
                if "is-unavailable" not in button.get_attribute("class"):
                    # Find the corresponding label text
                    label = button.find_element(By.XPATH, "./following-sibling::label/span[@class='opt-label__text']")
                    sizes.append(label.text.strip())

            sizes = ",".join(sizes)
            print("Available Sizes:", sizes)

            available = 0 if sizes == "" else 1
            print(f"Available: {"Yes" if available == 1 else "No"}")

            image_element = driver.find_element(By.CSS_SELECTOR, "a.show-gallery")
            image_url = image_element.get_attribute("href")
            print(image_url)
            print()
            # Append data
            products.append({
                "brand": brand,
                "title": title,
                "product_url": product_url,
                "regular_price": regular_price,
                "sale_price":sale_price,
                "description": description,
                "sizes": sizes,
                "available": available,
                "image_url": image_url
            })

            # Go back and wait for products page to reload
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block-inner-inner")))

        except Exception as e:
            print(f"Error scraping a product: {e}")
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block-inner-inner")))
            continue
    page_number+=1
    # Try to find the "Next" button
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.pagination__next")
    except:
        print("No more pages. Stopping.")
        break  


# Close Selenium WebDriver
driver.quit()

# Save to CSV

csv_file = "hedomak_products.csv"

# Convert products list to DataFrame
df = pd.DataFrame(products)

# Check if the file exists
if os.path.exists(csv_file):
    # Append without writing headers again
    df.to_csv(csv_file, mode='a', header=False, index=False)
else:
    # Write new file with headers
    df.to_csv(csv_file, mode='w', header=True, index=False)

print("Scraping complete. Data saved!")
